# MD Simulation Test - CPU Only (Steps 1-8)

Test instalasi dan setup topology menggunakan **ACPYPE** untuk proper GROMACS ligand topology.

**Features:**
- Micromamba environment caching (skip install if cache exists)
- Checkpoint saving per step for resume capability

**Jalankan dengan Accelerator: None**

---

In [ ]:
# Config
import os
import shutil
from pathlib import Path

COMPLEX_NAME = '264THM_PPARG'
WORK_DIR = f'/kaggle/working/{COMPLEX_NAME}'
TOPOL_DIR = f'{WORK_DIR}/topol'
OUTPUT_DIR = Path('/kaggle/working/output')
MAMBA_CACHE_DATASET = Path('/kaggle/input/mamba-amber-cache')

# Create output directory for checkpoints
OUTPUT_DIR.mkdir(exist_ok=True)

print(f'Testing: {COMPLEX_NAME}')
print(f'Output dir: {OUTPUT_DIR}')

## Step 1: Install GROMACS

In [ ]:
%%bash
set -e
apt-get update -qq
apt-get install -qq gromacs
gmx --version | head -3
echo 'GROMACS: OK'

## Step 2: Install Micromamba + AmberTools + ACPYPE

**With Caching:** If cache dataset exists, extract from cache. Otherwise install fresh and save to output.

In [ ]:
# Check for cached mamba environment
import tarfile

MAMBA_ROOT = Path('/kaggle/working/mamba')
MAMBA_BIN = Path('/kaggle/working/bin/micromamba')
CACHE_TAR = MAMBA_CACHE_DATASET / 'mamba-amber-cache.tar.gz'

if CACHE_TAR.exists():
    print('=== CACHE FOUND! Extracting... ===')
    with tarfile.open(CACHE_TAR, 'r:gz') as tar:
        tar.extractall('/kaggle/working')
    print(f'Extracted cache to /kaggle/working')
    USE_CACHE = True
else:
    print('=== No cache found. Will install fresh. ===')
    USE_CACHE = False

In [ ]:
%%bash
set -e

# Check if already extracted from cache
if [ -f /kaggle/working/bin/micromamba ]; then
    echo 'Micromamba already exists (from cache)'
    /kaggle/working/bin/micromamba --version
    echo 'Micromamba: OK (cached)'
else
    echo 'Installing micromamba fresh...'
    cd /kaggle/working
    curl -Ls https://micro.mamba.pm/api/micromamba/linux-64/latest | tar -xvj bin/micromamba
    ./bin/micromamba --version
    echo 'Micromamba: OK (fresh install)'
fi

In [ ]:
%%bash
set -e

export MAMBA_ROOT_PREFIX=/kaggle/working/mamba

# Check if environment already exists
if [ -d "/kaggle/working/mamba/envs/amber" ]; then
    echo 'Amber environment already exists (from cache)'
    echo 'AmberTools + ACPYPE: OK (cached)'
else
    echo 'Creating amber environment fresh...'
    cd /kaggle/working
    ./bin/micromamba create -n amber -c conda-forge ambertools=23 openbabel acpype python=3.10 -y
    echo 'AmberTools + ACPYPE: OK (fresh install)'
fi

In [ ]:
%%bash
set -e
cd /kaggle/working
export MAMBA_ROOT_PREFIX=/kaggle/working/mamba
eval "$(./bin/micromamba shell hook -s bash)"
micromamba activate amber
echo "antechamber: $(which antechamber)"
echo "acpype: $(which acpype)"
echo 'All tools verified!'

In [ ]:
# Save mamba environment to output for future caching (if fresh install)
import tarfile

if not USE_CACHE:
    print('=== Saving mamba environment to output for caching ===')
    cache_output = OUTPUT_DIR / 'mamba-amber-cache.tar.gz'
    with tarfile.open(cache_output, 'w:gz') as tar:
        tar.add('/kaggle/working/bin', arcname='bin')
        tar.add('/kaggle/working/mamba', arcname='mamba')
    print(f'Cache saved to: {cache_output}')
    print(f'Size: {cache_output.stat().st_size / 1024 / 1024:.1f} MB')
    print('\n>>> IMPORTANT: Upload this file as a Kaggle Dataset for future runs! <<<')
else:
    print('Using cached environment, no need to save.')

## Step 3: Setup Working Directory

In [ ]:
work_dir = Path(WORK_DIR)
topol_dir = Path(TOPOL_DIR)
for d in ['input', 'topol', 'em']:
    (work_dir / d).mkdir(parents=True, exist_ok=True)
print(f'Working: {work_dir}')

In [ ]:
# Copy input files
DATASET_DIR = Path('/kaggle/input/md-simulation-264thm-pparg-input')
if DATASET_DIR.exists():
    for f in DATASET_DIR.glob('*.pdb'):
        shutil.copy(f, work_dir / 'input')
        print(f'Copied: {f.name}')
else:
    raise FileNotFoundError(f'Dataset not found: {DATASET_DIR}')

## Step 4: Protein Topology (pdb2gmx)

In [ ]:
%%bash
set -e
cd /kaggle/working/264THM_PPARG/topol
printf '1\n' | gmx pdb2gmx -f ../input/PPARG_6MS7.pdb -o protein.gro -p topol.top -i posre.itp -ff amber99sb-ildn -water tip3p -ignh
echo 'Protein topology: OK'

## Step 5: Ligand Topology (ACPYPE with GAFF2)

ACPYPE generates proper GROMACS `.itp` files with correct atomtypes and moleculetype sections.

In [ ]:
%%bash
set -e
cd /kaggle/working
export MAMBA_ROOT_PREFIX=/kaggle/working/mamba
eval "$(./bin/micromamba shell hook -s bash)"
micromamba activate amber

cd /kaggle/working/264THM_PPARG/topol

# Run ACPYPE with GAFF2 force field
# -i: input file
# -n 0: neutral charge
# -a gaff2: use GAFF2 force field
# -o gmx: output for GROMACS
acpype -i ../input/264-trihydroxy-4-methoxybenzophenone_docked.pdb -n 0 -a gaff2 -o gmx

echo '\n=== ACPYPE OUTPUT FILES ==='
ls -la *.acpype/
echo 'ACPYPE: OK'

In [ ]:
%%bash
set -e
cd /kaggle/working/264THM_PPARG/topol

# Find ACPYPE output directory
ACPYPE_DIR=$(ls -d *.acpype 2>/dev/null | head -1)
if [ -z "$ACPYPE_DIR" ]; then
    echo 'ERROR: ACPYPE output directory not found!'
    exit 1
fi

echo "ACPYPE dir: $ACPYPE_DIR"

# Copy GROMACS files from ACPYPE output
# ACPYPE outputs: *_GMX.gro, *_GMX.itp, *_GMX.top
cp $ACPYPE_DIR/*_GMX.gro ligand.gro
cp $ACPYPE_DIR/*_GMX.itp ligand.itp

# Show first 30 lines of ligand.itp to verify structure
echo '\n=== ligand.itp first 30 lines ==='
head -30 ligand.itp

echo '\nLigand files copied: OK'

## Step 6: Combine Protein + Ligand

In [ ]:
# Combine GRO files
os.chdir(topol_dir)

with open('protein.gro', 'r') as f:
    protein_lines = f.readlines()
with open('ligand.gro', 'r') as f:
    ligand_lines = f.readlines()

protein_atoms = protein_lines[2:-1]
ligand_atoms = ligand_lines[2:-1]
box = protein_lines[-1]
total_atoms = len(protein_atoms) + len(ligand_atoms)

with open('complex.gro', 'w') as f:
    f.write(f'{COMPLEX_NAME} complex\n')
    f.write(f' {total_atoms}\n')
    f.writelines(protein_atoms)
    f.writelines(ligand_atoms)
    f.write(box)

print(f'Complex: {total_atoms} atoms')

In [ ]:
# Update topology file
# GROMACS topology order:
# 1. #include forcefield.itp
# 2. #include ligand.itp (contains atomtypes + moleculetype)
# 3. Protein moleculetype
# 4. [ system ]
# 5. [ molecules ]

with open('topol.top', 'r') as f:
    topol = f.read()

# Find forcefield include line
ff_pattern = '#include "amber99sb-ildn.ff/forcefield.itp"'
ff_pos = topol.find(ff_pattern)
if ff_pos == -1:
    raise ValueError('Cannot find forcefield include!')

ff_end = topol.find('\n', ff_pos) + 1

# Insert ligand.itp AFTER forcefield
# ACPYPE's ITP contains both atomtypes and moleculetype in correct order
ligand_include = '\n; Include ligand topology (ACPYPE/GAFF2)\n#include "ligand.itp"\n'
new_topol = topol[:ff_end] + ligand_include + topol[ff_end:]

# Get ligand residue name from ligand.itp
with open('ligand.itp', 'r') as f:
    itp_content = f.read()

# Find moleculetype name
import re
moltype_match = re.search(r'\[ moleculetype \]\s*\n;.*\n\s*(\S+)', itp_content)
if moltype_match:
    lig_name = moltype_match.group(1)
else:
    lig_name = 'LIG'
print(f'Ligand moleculetype name: {lig_name}')

# Add ligand to [ molecules ] section
if lig_name not in new_topol.split('[ molecules ]')[-1]:
    new_topol += f'\n{lig_name}     1\n'

with open('topol.top', 'w') as f:
    f.write(new_topol)

print('Topology updated!')
print('\n--- First 35 lines of topol.top ---')
with open('topol.top', 'r') as f:
    for i, line in enumerate(f):
        if i < 35:
            print(f'{i+1:3}: {line}', end='')
        else:
            break

## Step 7: Solvate + Ions

In [ ]:
%%bash
set -e
cd /kaggle/working/264THM_PPARG/topol
gmx editconf -f complex.gro -o box.gro -c -d 1.2 -bt dodecahedron
gmx solvate -cp box.gro -cs spc216.gro -o solvated.gro -p topol.top
echo 'Solvated: OK'

In [ ]:
# Create ions.mdp
ions_mdp = '''integrator = steep
emtol = 1000.0
emstep = 0.01
nsteps = 50000
nstlist = 10
cutoff-scheme = Verlet
coulombtype = cutoff
rcoulomb = 1.0
rvdw = 1.0
pbc = xyz
'''
with open('ions.mdp', 'w') as f:
    f.write(ions_mdp)
print('ions.mdp created')

In [ ]:
%%bash
set -e
cd /kaggle/working/264THM_PPARG/topol
gmx grompp -f ions.mdp -c solvated.gro -p topol.top -o ions.tpr -maxwarn 2
printf 'SOL\n' | gmx genion -s ions.tpr -o system.gro -p topol.top -pname NA -nname CL -neutral -conc 0.15
echo 'Ions: OK'

## Step 8: Energy Minimization (CPU)

In [ ]:
os.chdir(work_dir)
(work_dir / 'em').mkdir(exist_ok=True)

em_mdp = '''integrator = steep
emtol = 1000.0
emstep = 0.01
nsteps = 50000
nstlist = 10
cutoff-scheme = Verlet
coulombtype = PME
rcoulomb = 1.0
rvdw = 1.0
pbc = xyz
'''
with open('em/em.mdp', 'w') as f:
    f.write(em_mdp)
print('em.mdp created')

In [ ]:
%%bash
set -e
cd /kaggle/working/264THM_PPARG
gmx grompp -f em/em.mdp -c topol/system.gro -p topol/topol.top -o em/em.tpr -maxwarn 2
gmx mdrun -v -deffnm em/em
echo '\n=== ENERGY MINIMIZATION COMPLETE ===' 
echo 'Step 8: OK'

## Save Checkpoint: All Setup Files

In [ ]:
# Save all setup files as checkpoint
import tarfile

checkpoint_dir = OUTPUT_DIR / 'checkpoint_setup'
checkpoint_dir.mkdir(exist_ok=True)

# Copy important files
files_to_save = [
    ('topol/topol.top', 'topol.top'),
    ('topol/posre.itp', 'posre.itp'),
    ('topol/ligand.itp', 'ligand.itp'),
    ('topol/system.gro', 'system.gro'),
    ('em/em.gro', 'em.gro'),
    ('em/em.edr', 'em.edr'),
    ('em/em.log', 'em.log'),
]

for src, dst in files_to_save:
    src_path = work_dir / src
    if src_path.exists():
        shutil.copy(src_path, checkpoint_dir / dst)
        print(f'Saved: {dst}')

# Create compressed archive
archive_path = OUTPUT_DIR / f'{COMPLEX_NAME}_setup_checkpoint.tar.gz'
with tarfile.open(archive_path, 'w:gz') as tar:
    tar.add(checkpoint_dir, arcname='setup')
print(f'\nCheckpoint archive: {archive_path}')
print(f'Size: {archive_path.stat().st_size / 1024 / 1024:.2f} MB')

print('\n>>> Upload this checkpoint to Kaggle Dataset to resume from Step 9! <<<')

---

## SUCCESS!

Jika sampai sini tidak ada error, berarti setup sudah benar.

**Output files:**
1. `mamba-amber-cache.tar.gz` - Cache micromamba environment (upload as dataset)
2. `264THM_PPARG_setup_checkpoint.tar.gz` - Setup files checkpoint

**Next:** Run notebook utama dengan GPU untuk production MD (Steps 9-11).